<a href="https://colab.research.google.com/github/programminghumanity/iphs200fall2020/blob/main/twitter_step1_search_combine_20201128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Gathering Tweets**

References:

* https://www.youtube.com/watch?v=_SqgSh3aR1g (20201127 10:17)
* https://null-byte.wonderhowto.com/how-to/mine-twitter-for-targeted-information-with-twint-0193853/ (20190502) 
* https://github.com/twintproject/twint/wiki (wiki)
* https://pielco11.ovh/posts/twint-osint/ (osint)

# **0. Setup Environment**

## You will need to give permission for this Colab to link to your gdrive in the code cell below

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
# CUSTOMIZE: if you want your work and twitter datasets saved into a specific folder
#            beneath your gdrive root directory, define it below

dir_working = './MyDrive/courses/2020f_iphs200_programming_humanity/code/'

In [3]:
# Move to working directory

%cd $dir_working

/gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code


In [4]:
!git clone --depth=1 https://github.com/twintproject/twint.git


fatal: destination path 'twint' already exists and is not an empty directory.


In [5]:
%cd twint


/gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code/twint


In [6]:
# RESTART RUNTIME button at end of output (this takes a bit of time)

!pip3 install . -r requirements.txt

Processing /gdrive/MyDrive/courses/2020f_iphs200_programming_humanity/code/twint
     |████████████████████████████████| 1.3MB 5.3MB/s 
     |████████████████████████████████| 266kB 27.1MB/s 
     |████████████████████████████████| 327kB 28.6MB/s 
     |████████████████████████████████| 143kB 33.8MB/s 
     |████████████████████████████████| 296kB 34.7MB/s 
     |████████████████████████████████| 235kB 40.7MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=792da676905eb2bbfd08d60e77ee95f2d559a2e707632a74d420c9ae8a7a8626
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for googletransx: filename=googletransx-2.4.2-cp36-none-any.whl size=15970 sha256=feeaa772f655008ea045386376795637be66079eebb49af50adeb136ce4fc647
  Stored in directory: /root/.cache/pip/wheels/04/63/5f/75e7e94eb62517946116a783e4cd8970c478

## You should [RESTART RUNTIME] at this point before continuing

### WORKAROUND to TWINT ISSUE No.1061 introduced 15 Dec 2020

Reference: https://github.com/twintproject/twint/issues/1061

In [14]:
!pip uninstall twint

Uninstalling twint-2.1.21:
  Would remove:
    /usr/local/bin/twint
    /usr/local/lib/python3.6/dist-packages/twint-2.1.21.dist-info/*
    /usr/local/lib/python3.6/dist-packages/twint/*
Proceed (y/n)? y
  Successfully uninstalled twint-2.1.21


In [15]:
!pip install git+git://github.com/ajctrl/twint@patch-1

  Cloning git://github.com/ajctrl/twint (to revision patch-1) to /tmp/pip-req-build-ip1nson5
  Running command git clone -q git://github.com/ajctrl/twint /tmp/pip-req-build-ip1nson5
  Running command git checkout -b patch-1 --track origin/patch-1
  Switched to a new branch 'patch-1'
  Branch 'patch-1' set up to track remote branch 'patch-1' from 'origin'.
  Created wheel for twint: filename=twint-2.1.21-cp36-none-any.whl size=38761 sha256=fd56e938095d326fb7ae938514963fc06b59ca0f6c9bac5c3a314732751a9a9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-_37rca2k/wheels/51/ee/ba/293fdc533d7a474a69eeebf72eaa1c6ff1b2a20236eebe6f16
Successfully built twint


## You should [RESTART RUNTIME] at this point before continuing

In [12]:
# Not necessary, but we do lose all other variables and configuration with [RESTART RUNTIME] in last cell
#                so the following cells re-setup our configuration

"""
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive
"""

"\nfrom google.colab import drive\ndrive.mount('/gdrive')\n%cd /gdrive\n"

In [1]:
# CUSTOMIZE: if you want your work and twitter datasets saved into a specific folder
#            beneath your gdrive root directory, define it below

dir_working = './MyDrive/courses/2020f_iphs200_programming_humanity/code/'

In [2]:
# Move to working directory

%cd $dir_working

[Errno 2] No such file or directory: './MyDrive/courses/2020f_iphs200_programming_humanity/code/'
/content


In [3]:
# Move into the twint subdirectory created with the installtion of twint

%cd twint

[Errno 2] No such file or directory: 'twint'
/content


In [4]:
!pwd

/content


In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
import os
import glob

import time

In [7]:
import twint

In [8]:
import nest_asyncio
nest_asyncio.apply()

# **1. DEFINE and RUN Searches**

In [9]:
# LEAVE this blank, this is just defining a global
#   list variable that we'll populate below

search_terms_ls = [] # LEAVE THIS EMPTY !!!

## (a). Define your search_terms and/or user_name in the following code cell

In [10]:
# STEP ONE: DEFINE search

# Define your search terms and/or user (you must set values for at least one of these, both ok too)

# CUSTOMIZE: If you want to limit your search to a particular user set 'username' to it
user_name = 'mtbarra' # uncomment this if you want to search by username (only one username, no spaces)
# user_name = '' # uncomment this line if you are not searching by username, only search terms

# CUSTOMIZE: If you want to search for key terms in the tweet, set 'search_terms'
search_terms = '' # uncomment this if you want to search by key words, format 'term1 term2 term3'
                       #   all search terms should be separated by a single space within one string
# search_terms = '' # uncomment this line if you are not searching by any search terms, only username

In [17]:
# STEP TWO: RUN search

# Create a filename of search results in a file with the name contained a unique timestamp
timestr = time.strftime("%Y%m%d-%H%M%S")

"""
if len(search_terms) == 0:
  search_str = ''
else:
  search_str = '_'.join(search_terms.split())

if len(user_name) == 0:
  user_str = ''
else:
  user_str = user_name
"""

file_name = f"tweets_twint_{search_terms}_{user_name}_{timestr}.csv"
print(f"FILENAME: {file_name}")

# Save your user/search terms
search_terms_both = f"USERNAME:{user_name} and SEARCHTERMS:{search_terms}, "
search_terms_ls.append(search_terms_both)

# Run your search
c = twint.Config()
if len(user_name) > 0:
  c.Username = user_name
if len(search_terms) > 0:
  c.Search = search_terms

if (len(user_name) != 0)  | (len(search_terms) != 0):
  c.Limit = 10000    # Limit of how many tweets to retrieve
  c.Store_object=True
  c.Store_csv = True
  c.Output = file_name # "tweets.csv"
  twint.run.Search(c)
else:
  print("You need to set user_name and/or search_terms to a non-null value")

FILENAME: tweets_twint_GM_mtbarra_20201217-035634.csv
1330901438298525698 2020-11-23 15:50:04 +0000 <mtbarra> We have reached an important inflection point at @GM. We are transitioning to an all-electric portfolio, and doing so from a position of unprecedented strength. This is how we will win.  https://t.co/9HeJlntI9v
1325910961715339264 2020-11-09 21:19:42 +0000 <mtbarra> Congratulations to @JimmieJohnson as he retires with the honorable legacy of 7-time NASCAR Cup series champion. Thank you for making our teams at @GM and @Chevrolet so proud.
1322272268781846529 2020-10-30 20:20:50 +0000 <mtbarra> One of the best CFOs in the world is joining @GM.  Welcome to the team, Paul Jacobson.
1320001660169736197 2020-10-24 13:58:14 +0000 <mtbarra> @fatma_samoura @GM Thank you, @Fatma_Samoura. It’s an honor to be inducted alongside both you and @GinniRometty. Congratulations on the well-deserved recognition for your leadership.
1318723212231036931 2020-10-21 01:18:09 +0000 <mtbarra> What an ex

## (b). Go back and run the last two code cells DEFINE SEARCH and RUN SEARCH until:

* You modify your searches to focus in on what you want to find out
* You have gathered enough tweets necessary for rigourous analysis

In [18]:
!pwd

/content


In [19]:
!ls -altr *.csv

-rw-r--r-- 1 root root 76692 Dec 17 03:53 tweets_twint_GM_mtbarra_20201217-035303.csv
-rw-r--r-- 1 root root 76692 Dec 17 03:56 tweets_twint_GM_mtbarra_20201217-035621.csv
-rw-r--r-- 1 root root 76692 Dec 17 03:56 tweets_twint_GM_mtbarra_20201217-035634.csv


In [22]:
!ls -altr tweets_twint*

-rw-r--r-- 1 root root 76692 Dec 17 03:53 tweets_twint_GM_mtbarra_20201217-035303.csv
-rw-r--r-- 1 root root 76692 Dec 17 03:56 tweets_twint_GM_mtbarra_20201217-035621.csv
-rw-r--r-- 1 root root 76692 Dec 17 03:56 tweets_twint_GM_mtbarra_20201217-035634.csv


In [23]:
# COPY AND PASTE the name of the datafile listed above into command below

tweets_df = pd.read_csv('tweets_twint_GM_mtbarra_20201217-035634.csv') # PASTE the correct tweets_twint_*.csv filename from the output of the last cell

In [25]:
# Try converting DataFrame.Series/columns to more specific types

tweets_df = tweets_df.convert_dtypes()
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               131 non-null    Int64  
 1   conversation_id  131 non-null    Int64  
 2   created_at       131 non-null    string 
 3   date             131 non-null    string 
 4   time             131 non-null    string 
 5   timezone         131 non-null    Int64  
 6   user_id          131 non-null    Int64  
 7   username         131 non-null    string 
 8   name             131 non-null    string 
 9   place            0 non-null      Int64  
 10  tweet            131 non-null    string 
 11  language         131 non-null    string 
 12  mentions         131 non-null    string 
 13  urls             131 non-null    string 
 14  photos           131 non-null    string 
 15  replies_count    131 non-null    Int64  
 16  retweets_count   131 non-null    Int64  
 17  likes_count     

In [26]:
tweets_df.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1330901438298525698,1330901438298525698,2020-11-23 15:50:04 UTC,2020-11-23,15:50:04,0,1219824320,mtbarra,Mary Barra,<NA>,We have reached an important inflection point ...,en,"[{'screen_name': 'gm', 'name': 'general motors...",[],[],554,178,884,[],[],https://twitter.com/mtbarra/status/13309014382...,False,<NA>,1,https://pbs.twimg.com/ext_tw_video_thumb/13309...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>
1,1325910961715339264,1325910961715339264,2020-11-09 21:19:42 UTC,2020-11-09,21:19:42,0,1219824320,mtbarra,Mary Barra,<NA>,Congratulations to @JimmieJohnson as he retire...,en,"[{'screen_name': 'jimmiejohnson', 'name': 'jim...",[],[],21,7,93,[],[],https://twitter.com/mtbarra/status/13259109617...,False,https://twitter.com/TeamChevy/status/132580031...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>
2,1322272268781846529,1322272268781846529,2020-10-30 20:20:50 UTC,2020-10-30,20:20:50,0,1219824320,mtbarra,Mary Barra,<NA>,One of the best CFOs in the world is joining @...,en,"[{'screen_name': 'gm', 'name': 'general motors...",[],[],536,39,351,[],[],https://twitter.com/mtbarra/status/13222722687...,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>
3,1320001660169736197,1319720932211130370,2020-10-24 13:58:14 UTC,2020-10-24,13:58:14,0,1219824320,mtbarra,Mary Barra,<NA>,"@fatma_samoura @GM Thank you, @Fatma_Samoura. ...",en,"[{'screen_name': 'fatma_samoura', 'name': 'fat...",[],[],3,1,11,[],[],https://twitter.com/mtbarra/status/13200016601...,False,<NA>,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"[{'screen_name': 'fatma_samoura', 'name': 'Fat...",<NA>,<NA>,<NA>,<NA>
4,1318723212231036931,1318723212231036931,2020-10-21 01:18:09 UTC,2020-10-21,01:18:09,0,1219824320,mtbarra,Mary Barra,<NA>,"What an exciting night! The design, engineerin...",en,"[{'screen_name': 'gm', 'name': 'general motors...",[],[],77,78,431,"['gmchummerev', 'ev']",[],https://twitter.com/mtbarra/status/13187232122...,False,https://twitter.com/GMC/status/131872142044106...,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[],<NA>,<NA>,<NA>,<NA>


# **2. Aggregate your twitter datasets from all your searches and write to file (including a manifest file)**

In [28]:
!ls tweets_twint*

tweets_twint_GM_mtbarra_20201217-035303.csv
tweets_twint_GM_mtbarra_20201217-035621.csv
tweets_twint_GM_mtbarra_20201217-035634.csv


In [29]:
# CUSTOMIZE: Define a RegEx pattern that matches all the twitter dataset files 
#            you want to combine (e.g. tweets_twint_*.csv)

tweet_filenames = 'tweets_twint_*.csv'  # Define the RegEx that matches all your twitter dataset files

all_tweet_files = [i for i in glob.glob(tweet_filenames)] # You should use 'tweets_*.csv'
print(all_tweet_files)

['tweets_twint_GM_mtbarra_20201217-035634.csv', 'tweets_twint_GM_mtbarra_20201217-035303.csv', 'tweets_twint_GM_mtbarra_20201217-035621.csv']


In [30]:
# Review user/search terms used to create these files

search_terms_ls

['USERNAME:mtbarra and SEARCHTERMS:GM',
 'USERNAME:mtbarra and SEARCHTERMS:GM',
 'USERNAME:mtbarra and SEARCHTERMS:GM']

In [31]:
# Combine all twitter dataset files into one pd.DataFrame
combined_df = pd.concat([pd.read_csv(f) for f in all_tweet_files ])

# Create a filename of search results in a file with the name contained a unique timestamp
timestr = time.strftime("%Y%m%d-%H%M%S")
file_name_combo = f"tweets_combined_{timestr}.csv"

# Write all tweets combined into one big file
combined_df.to_csv(file_name_combo, index=False, encoding='utf-8-sig')

# Write an accompnaying manifest file with all the search terms used to gather tweets in the combined file
file_name_combo_root = file_name_combo.split('.')[0]
file_name_combo_manifest = f"{file_name_combo_root}_manifest.txt"
with open(file_name_combo_manifest, 'w') as fp:
  fp.writelines(search_terms_ls)

In [34]:
# CONFIRM: Both the 'tweets_combined_*.csv' and 'tweets_combined_*_manifest.txt' files exist
#          and that the size of the combined dataset file is approximately equal to all the 
#          individual dataset files that went into it

!ls -altr tweets_*

-rw-r--r-- 1 root root  76692 Dec 17 03:53 tweets_twint_GM_mtbarra_20201217-035303.csv
-rw-r--r-- 1 root root  76692 Dec 17 03:56 tweets_twint_GM_mtbarra_20201217-035621.csv
-rw-r--r-- 1 root root  76692 Dec 17 03:56 tweets_twint_GM_mtbarra_20201217-035634.csv
-rw-r--r-- 1 root root 227491 Dec 17 03:58 tweets_combined_20201217-035841.csv
-rw-r--r-- 1 root root    105 Dec 17 03:58 tweets_combined_20201217-035841_manifest.txt


In [35]:
# CHECK: Make sure the manifest.txt file associated with our combined tweet dataset file saved all our
#        search terms

# CONFIGURE: cut-n-paste in the full filename of the most recent '*_manisfest.txt' file listed in the previous code cell above

!cat tweets_combined_20201217-035841_manifest.txt

USERNAME:mtbarra and SEARCHTERMS:GMUSERNAME:mtbarra and SEARCHTERMS:GMUSERNAME:mtbarra and SEARCHTERMS:GM

In [36]:
combined_df = combined_df.convert_dtypes()
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 393 entries, 0 to 130
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               393 non-null    Int64  
 1   conversation_id  393 non-null    Int64  
 2   created_at       393 non-null    string 
 3   date             393 non-null    string 
 4   time             393 non-null    string 
 5   timezone         393 non-null    Int64  
 6   user_id          393 non-null    Int64  
 7   username         393 non-null    string 
 8   name             393 non-null    string 
 9   place            0 non-null      Int64  
 10  tweet            393 non-null    string 
 11  language         393 non-null    string 
 12  mentions         393 non-null    string 
 13  urls             393 non-null    string 
 14  photos           393 non-null    string 
 15  replies_count    393 non-null    Int64  
 16  retweets_count   393 non-null    Int64  
 17  likes_count     

In [37]:
!pwd

/content


In [38]:
!ls

sample_data
tweets_combined_20201217-035841.csv
tweets_combined_20201217-035841_manifest.txt
tweets_twint_GM_mtbarra_20201217-035303.csv
tweets_twint_GM_mtbarra_20201217-035621.csv
tweets_twint_GM_mtbarra_20201217-035634.csv


# **3. Clean tweets and separate out subcomponents (e.g. hashtags, emojis, etc)**

# **4. Exploratory Data Analysis (EDA) and Visualizations of Raw Data**